In [1]:
from diel_models.pipeline import Pipeline
from diel_models.pipeline_intermediate import DayNightCreator, BiomassRegulator, NitrateUptakeRatioCalibrator, StoragePoolGenerator, PhotonReactionInhibitor
import os
import cobra
from tests import TEST_DIR

aragem_model_path = os.path.join(TEST_DIR, 'data', 'AraGEM2010.xml')

modelo = cobra.io.read_sbml_model(aragem_model_path)

Steps = [
    DayNightCreator(modelo),
    StoragePoolGenerator(modelo, metabolites = ["S_Sucrose_c_Day", "S_Sulfate_c_Day", "S_Nitrate_c_Day", "S_L_45_Histidine_c_Day", "S_L_45_Isoleucine_c_Day", "S_L_45_Leucine_c_Day", "S_L_45_Lysine_c_Day", "S_L_45_Methionine_c_Day", "S_L_45_Phenylalanine_c_Day", "S_L_45_Threonine_c_Day", "S_L_45_Tryptophan_c_Day", "S_L_45_Valine_c_Day", "S_L_45_Arginine_c_Day", "S_L_45_Cysteine_c_Day", "S_L_45_Glutamine_c_Day", "S_L_45_Glutamate_c_Day", "S_Glycine_c_Day", "S_L_45_Proline_c_Day", "S_L_45_Tyrosine_c_Day", "S_L_45_Alanine_c_Day", "S_L_45_Asparagine_c_Day", "S_L_45_Serine_c_Day", "S_Orthophosphate_c_Day", "S_Starch_p_Day", "S_D_45_Fructose_c_Day", "S__40_S_41__45_Malate_c_Day", "S_Fumarate_c_Day", "S_Citrate_c_Day"]),
    PhotonReactionInhibitor(modelo, "Ex16_Night"),
    BiomassRegulator(modelo, "BIO_L_Day", "BIO_L_Night", ["R03845_c_Night", "R06267_c_Night", "R06286_c_Night", "R04801_c_Night", "R07856_c_Night", "R04803_c_Night", "R06961_c_Night", "R06960_c_Night", "V0004_Night", "R06284_c_Night", "R05618_c_Night", "R03824_c_Night", "R05341_c_Night", "R06965_c_Night", "R07558_c_Night", "R04798_c_Night", "R04787_c_Night", "R07840_c_Night", "R06963_c_Night", "R06962_c_Night", "R07850_c_Night", "R07530_c_Night"]),
    NitrateUptakeRatioCalibrator(modelo, "Ex4_Day", "Ex4_Night")
]

pipeline = Pipeline(modelo, Steps)
pipeline.run()

In [10]:
#modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
#modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
#modelo.objective = "Ex16_Day"
#modelo.objective_direction = "max"

In [17]:
#modelo.reactions.get_by_id("Biomass_Total").upper_bound = 1000
#modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0
#modelo.objective = "Ex16_Day"
#modelo.objective_direction = "max"

In [24]:
#modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
#modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
#modelo.objective = "Biomass_Total"
#modelo.objective_direction = "max"

In [31]:
modelo.reactions.get_by_id("Biomass_Total").upper_bound = 1000
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0
modelo.objective = "Biomass_Total"
modelo.objective_direction = "max"

In [32]:
from cobra.flux_analysis import pfba

solution = pfba(modelo)
R02732_c = solution.fluxes["R02732_c_Day"] + solution.fluxes["R02732_c_Night"]
R01070N_c = solution.fluxes["R01070N_c_Day"] + solution.fluxes["R01070N_c_Night"]
R00762_c = solution.fluxes["R00762_c_Day"] + solution.fluxes["R00762_c_Night"]

chex2 = round(((R02732_c - R01070N_c) + R00762_c) / 2, 2)
print(chex2)

9.88


In [33]:
R04779_p = solution.fluxes["R04779_p_Day"] + solution.fluxes["R04779_p_Night"]
R00762_p = solution.fluxes["R00762_p_Day"] + solution.fluxes["R00762_p_Night"]
R01070N_p = solution.fluxes["R01070N_p_Day"] + solution.fluxes["R01070N_p_Night"]


phex1 = round(((R04779_p - R00762_p) - R01070N_p) / 2, 2)
phex1

0.0

In [34]:
R01061_c = solution.fluxes["R01061_c_Day"] + solution.fluxes["R01061_c_Night"]
R01061_p = solution.fluxes["R01061_p_Day"] + solution.fluxes["R01061_p_Night"]
R01063_p = solution.fluxes["R01063_p_Day"] + solution.fluxes["R01063_p_Night"]
R01512_c = solution.fluxes["R01512_c_Day"] + solution.fluxes["R01512_c_Night"]
R01518_c = solution.fluxes["R01518_c_Day"] + solution.fluxes["R01518_c_Night"]
R01518_p = solution.fluxes["R01518_p_Day"] + solution.fluxes["R01518_p_Night"]
R00658_c = solution.fluxes["R00658_c_Day"] + solution.fluxes["R00658_c_Night"]
R00658_p = solution.fluxes["R00658_p_Day"] + solution.fluxes["R00658_p_Night"]

Cphex = round(((R01061_c + R01061_p - R01063_p - R01512_c)/2 + (R01518_c + R01518_p) + (R00658_c + R00658_p))/3, 2)
Cphex #sem contar com R01512_d

40.03

In [35]:
R00200_c = solution.fluxes["R00200_c_Day"] + solution.fluxes["R00200_c_Night"]

chex3 = round(R00200_c,2)
chex3

44.04

In [36]:
R00200_p = solution.fluxes["R00200_p_Day"] + solution.fluxes["R00200_p_Night"]
R00206_p = solution.fluxes["R00206_p_Day"] + solution.fluxes["R00206_p_Night"]

phex2 = round(R00200_p - R00206_p, 4)
phex2

0.7129

In [37]:
R00351_m = solution.fluxes["R00351_m_Day"] + solution.fluxes["R00351_m_Night"]
R00209_m = solution.fluxes["R00209_m_Day"] + solution.fluxes["R00209_m_Night"]
TCM1 = solution.fluxes["TCM1_Day"] + solution.fluxes["TCM1_Night"]

tca2 = round((-TCM1 + R00351_m + R00209_m)/3, 2)
tca2

0.0

In [38]:
import pandas as pd

dados = {'Valores Artigo (pFBA)': ['11.38', '1.719', '19.07', '0.0065', '5.40', '9.51'],
         'Valores Artigo (MFA)' : ['7.33', '2.89', '23.96', '11.18', '7.95', '10.27'],
         'OF: max of Ex16_Day (pFBA)': ['9.27', '39.42', '39.51', '0.37', '0.0057', '0.01'],
         'OF: max of Ex16_Day (pFBA)*': ['9.85', '40.54', '40.59', '0.18', '0.0114', '0.0'],
         'OF: max of Biomass_Total (pFBA)' : ['0.02', '0.0', '0.34', '0.35', '0.0057', '0.0'],
         'OF: max of Biomass_Total (pFBA)*' : ['9.88', '0.0', '40.03', '44.04', '0.7129', '0.0']}

df = pd.DataFrame(dados)

nomes = ['chex2', 'phex1', 'Cphex', 'chex3', 'phex2', 'tca2']
df.index = nomes

df

,Valores Artigo (pFBA),Valores Artigo (MFA),OF: max of Ex16_Day (pFBA),OF: max of Ex16_Day (pFBA)*,OF: max of Biomass_Total (pFBA),OF: max of Biomass_Total (pFBA)*
chex2,11.38,7.33,9.27,9.85,0.02,9.88
phex1,1.719,2.89,39.42,40.53,0.0,0.0
Cphex,19.07,23.96,39.51,40.59,0.34,40.03
chex3,0.0065,11.18,0.37,0.18,0.35,44.04
phex2,5.40,7.95,0.0057,0.0114,0.0057,0.7129
tca2,9.51,10.27,0.01,0.0,0.0,0.0


*As constraints para a reação de Biomass_Total estão a (0, 1000) nestas hipóteses